## Load Solr with keyword facets

In our previous Solr index, had added features to do the following:

* Show facets along with search results to allow user to drill down into the subset of search results given by the facet.
* Allow better query parsing by using the keywords as a dictionary which can be looked up to isolate longer entities than just words.

In this instance, we will do the following:

* Add our newly mined ORG keywords as facets.

In addition, we will now focus on the content discovery side, adding features to do the following:

* Implement More Like This using title, abstract and text. This requires thee fields to have termVectors turned on.
* Implement Similar Documents functionality using keywords, authors and ORGs (similar to More Like This, except implementation reuses the facets we have already built).
* A more focused More Like This that uses a combined field kaoterms which stores multi-word terms as tokens. Internally MLT will use this field instead of title, abstract and text.


In [1]:
import json
import requests
import os
import sqlite3

In [2]:
DATA_DIR = "../data"

WORDCOUNTS_DB = os.path.join(DATA_DIR, "wordcounts.db")

CURATED_KEYWORDS = os.path.join(DATA_DIR, "raw_keywords.txt")
NEARDUP_MAPPINGS = os.path.join(DATA_DIR, "keyword_neardup_mappings.tsv")
DEDUPE_MAPPINGS = os.path.join(DATA_DIR, "keyword_dedupe_mappings.tsv")

TEXTFILES_DIR = os.path.join(DATA_DIR, "textfiles")
METADATA_FILE = os.path.join(DATA_DIR, "papers_metadata.tsv")

ORGS_DIR = os.path.join(DATA_DIR, "orgs")

SOLR_URL = "http://localhost:8983/solr/nips2index/"

In [3]:
conn = sqlite3.connect(WORDCOUNTS_DB)

### Index with keyword facets

First we create a new core to hold our new index.

    cd <solr_home>
    bin/solr create -c nips2index
    
Then add in our new schema. This has one additional multiValued string field keywords to hold our facets. Additionally we have changed the type of the authors field to "string" to allow faceting on authors as well.

    cd ../scripts
    ./create_config2.sh   # add MLT component
    ./create_schema2.sh


### Lookup tables for keywords

The full list of keywords is available in the `bigrams`, `rake` and `maui` tables. But they need to be filtered using the manually curated list of keywords. Finally these keywords need to be replaced with their canonical form so they look nicer when displayed as facets. 

We will also do a reverse lookup on these canonical form lookup tables when working on query expansion later.

In [4]:
valid_keywords = set()
fcurated = open(CURATED_KEYWORDS, "r")
for line in fcurated:
    valid_keywords.add(line.strip())
fcurated.close()

print("{:d} valid keywords".format(len(valid_keywords)))

2282 valid keywords


In [5]:
raw2canonical = {}
fneardup = open(NEARDUP_MAPPINGS, "r")
for line in fneardup:
    key, value = line.strip().split("\t")
    raw2canonical[key] = value
fneardup.close()

fdedupe = open(DEDUPE_MAPPINGS, "r")
for line in fdedupe:
    key, value, _ = line.strip().split("\t")
    raw2canonical[key] = value
fdedupe.close()

print("{:d} raw to canonical mappings".format(len(raw2canonical)))

455 raw to canonical mappings


In [6]:
def get_keywords_for_doc(conn, doc_id, valid_keywords, raw2canonical):
    keywords = set()
    # collect from bigrams
    cur_bigrams = conn.cursor()
    cur_bigrams.execute("select word_1, word_2 from bigrams where doc_id = ?", [doc_id])
    rows = cur_bigrams.fetchall()
    for row in rows:
        keyword = " ".join([row[0], row[1]]).lower()
        keywords.add(keyword)
    cur_bigrams.close()
    # collect from rake
    cur_rake = conn.cursor()
    cur_rake.execute("select keyword from rake where doc_id = ?", [doc_id])
    rows = cur_rake.fetchall()
    for row in rows:
        keywords.add(row[0].lower())
    cur_rake.close()
    # collect from maui
    cur_maui = conn.cursor()
    cur_maui.execute("select keyword from maui where doc_id = ?", [doc_id])
    rows = cur_maui.fetchall()
    for row in rows:
        keywords.add(row[0].lower())
    cur_maui.close()
    # filter out valid keywords
    filtered_keywords = [keyword for keyword in keywords 
                         if keyword in valid_keywords]
    # map them to their canonical forms if applicable
    canonical_keywords = set()
    for keyword in filtered_keywords:
        if keyword in raw2canonical.keys():
            canonical_keywords.add(raw2canonical[keyword])
        else:
            canonical_keywords.add(keyword)
    # return as list
    return list(canonical_keywords)


doc_id = 1
print(get_keywords_for_doc(conn, doc_id, valid_keywords, raw2canonical))

['total number', 'lower bound', 'ieee transactions', 'neural network', 'science foundation', 'hamming distance', 'neural net', 'upper bound', 'asymptotically optimal', 'maximum number', 'internal representation', 'matrix multiplication', 'optimal performance', 'information theory', 'random variables', 'computer science', 'national science foundation', 'associative memory', 'optimal values', 'national science']


### Document metadata lookup

We already saved the non-text values for each document in the `papers_metadata.tsv` file in the `01-preprocess` notebook, so we will use that here.

In [7]:
id2metadata = {}
fmeta = open(METADATA_FILE, "r")
for line in fmeta:
    line = line.strip()
    if line.startswith("#"):
        continue
    id, year, title, abstract, author_names = line.split("\t")
    authors = author_names.split(":")
    id2metadata[int(id)] = (year, title, abstract, authors)

fmeta.close()
print("{:d} metadata mappings".format(len(id2metadata)))

7238 metadata mappings


### ORG lookup

In [8]:
def get_orgs_for_doc(doc_id):
    orgs = []
    forg = open(os.path.join(ORGS_DIR, "{:d}.org".format(doc_id)), "r")
    for line in forg:
        orgs.append(line.strip())
    forg.close()
    return orgs


print(get_orgs_for_doc(978))

['University of Toronto', 'Aston University']


### KAO Terms builder

In [9]:
def build_kao_field(keywords, authors, orgs):
    kao_terms = []
    for multi_word_term in keywords + authors + orgs:
        kao_terms.append(multi_word_term.replace(" ", "0"))
    return " ".join(kao_terms)

print(build_kao_field(["neural network", "fitting models", "control points"],
                      ["Suguru Arimoto", "Phillip A. Chou"],
                      ["Osaka University" "CalTech"]))

neural0network fitting0models control0points Suguru0Arimoto Phillip0A.0Chou Osaka0UniversityCalTech


### Load papers into Solr

In [10]:
def count_index_rows(solr_url):
    resp = requests.get(solr_url + "select?q=*:*")
    resp_json = json.loads(resp.text)
    return resp_json["response"]["numFound"]


def add_row(solr_url, doc_id, conn, valid_keywords, raw2canonical, 
            id2metadata, should_commit):
    headers = {
        "content-type": "application/json",
        "accept": "application/json"
    }
    if doc_id is None:
        requests.post(solr_url + "update", params={"commit":"true"}, headers=headers)
    else:
        ftext = open(os.path.join(TEXTFILES_DIR, "{:d}.txt".format(doc_id)), "r")
        textfile_lines = []
        for line in ftext:
            textfile_lines.append(line.strip())
        ftext.close()
        text = "\n".join(textfile_lines)
        year, title, abstract, authors = id2metadata[doc_id]
        keywords = get_keywords_for_doc(conn, doc_id, valid_keywords, raw2canonical)
        orgs = get_orgs_for_doc(doc_id)
        kaoterms = build_kao_field(keywords, authors, orgs)
        req_body = json.dumps({
            "add": {
                "doc": {
                    "id": doc_id,
                    "year": year,
                    "title": title,
                    "abstract": abstract,
                    "text": text,
                    "authors": authors,
                    "keywords": keywords,
                    "orgs": orgs,
                    "kaoterms": kaoterms
                }
            }
        })
        params = { "commit": "true" if should_commit else "false" }
        requests.post(solr_url + "update", data=req_body, params=params, headers=headers)

In [11]:
num_rows_in_index = count_index_rows(SOLR_URL)
num_added = 0
should_commit = False
if num_rows_in_index == 0:
    for textfile in os.listdir(TEXTFILES_DIR):
        doc_id = int(textfile.split(".")[0])
        if num_added % 1000 == 0:
            print("{:d} records added".format(num_added))
            should_commit = True
        add_row(SOLR_URL, doc_id, conn, valid_keywords, raw2canonical, 
                id2metadata, should_commit)        
        should_commit = False
        num_added += 1
    
    print("{:d} records added, COMPLETE".format(num_added))
    add_row(SOLR_URL, None, conn, valid_keywords, raw2canonical, id2metadata, True)
    num_rows_in_index = count_index_rows(SOLR_URL)

print("{:d} records in index".format(num_rows_in_index))

0 records added
1000 records added
2000 records added
3000 records added
4000 records added
5000 records added
6000 records added
7000 records added
7238 records added, COMPLETE
7238 records in index
